___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Freddy Dratwa

#### Victor Vazquez

#### Michel Haggiag

#### Allan Singer

#### Ricardo Ferraro

___

# Estudo do preço do carro

___

## A. INTRODUÇÃO

## https://www.kaggle.com/jpayne/852k-used-car-listings

O objetivo deste projeto é chegar ao melhor modelo possível que seja capaz de prever o preço de um carro usado nos EUA a partir das características presentes no dataset:
    - Preço;
    - Ano;
    - Quilometragem;
    - Cidade;
    - Estado;
    - Marca;
    - Modelo;
    - Número de Chassis.

A base de dados possui 1.2 milhão de carros com data de produção entre os anos 1997 a 2018 de 58 marcas diferentes, sendo estes 2914 modelos distintos com o preço mÍnimo de US1.500,00 e máximo de US462.000,00 , com a mínima quilometragem 8 Km e a máxima 2.286.260,8 Km.<br>

Como uma primeira previsão, imagina-se que algumas das características do dataset influenciam mais no preço do carro do que outras. Por exemplo, é de se esperar que um carro comprado há mais tempo tenha seu preço menor atualmente. Ou seja, a variavel "Ano", que representa o ano em que o carro foi fabricado, deve influenciar muito no preço do veículo, bem como a quilometragem, onde um carro que foi comprado mas usado pouco deve permanecer mais barato do que um usado muitas vezes.<br>

Pelo outro lado, imagina-se que a variável "Cidade", a qual representa a cidade onde o carro foi comprado, não tenha tanta influência no preço do veículo, a não ser que comparadas  duas cidades com condições muito diferentes. Por exemplo, como se tratam de cidades apenas nos EUA, imaina-se que em cidades no litoral, os carros se depreciem com velocidade maior, como consequência do fenômeno da Maresia.<br>

___
## B. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

In [49]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

### BASE DE DADOS

**Limpando e tratando dataset**

Nas células abaixo, o dataset é limpo e reorganiazdo. Para se fazer isso, a coluna "Chassi" é removoida, pois esta é uma característica individual para cada veículo e não influencia no preço final do mesmo. Alem disso, duas colunas vazias foram reitradas. Então, a variável milhagem foi transformada em quilometragem, multplicando seu valor por 1.6. Por último, a variavel ano foi transformada em idade, ou seja, a diferença do ano atual (2019) para o ano em que o carro foi vendido. 

In [2]:
dataset = pd.read_csv("Dataframe.csv")

In [3]:
if "Vin" in dataset:
    dataset = dataset.drop(columns=["Vin", "Unnamed: 9", "Unnamed: 10", "Unnamed: 0"])
    dataset["Mileage"] = dataset["Mileage"].apply(lambda x: x*1.6)

    dataset.columns = ["ID","PRECO","ANO","KM","CIDADE","ESTADO","MARCA","MODELO"]
    dataset["IDADE"] = dataset["ANO"].apply(lambda x: 2019-x)

    dataset.ID = dataset.ID.astype("float")
    dataset.PRECO = dataset.PRECO.astype("float")
    dataset.ANO = dataset.ANO.astype("float")
    dataset.KM = dataset.KM.astype("float")
    dataset.IDADE = dataset.IDADE.astype("float")
    dataset.CIDADE = dataset.CIDADE.astype("category")
    dataset.ESTADO = dataset.ESTADO.astype("category")
    dataset.MARCA = dataset.MARCA.astype("category")
    dataset.MODELO = dataset.MODELO.astype("category")

    dataset["ESTADO"] = dataset["ESTADO"].apply(lambda x: x.upper())
    dataset["CIDADE"] = dataset["CIDADE"].apply(lambda x: x.upper())
    dataset["MARCA"] = dataset["MARCA"].apply(lambda x: x.upper())
    dataset = dataset[["ID","PRECO","ANO","IDADE","KM","CIDADE","ESTADO","MARCA","MODELO"]]
    
dataset.head()

,ID,PRECO,ANO,IDADE,KM,CIDADE,ESTADO,MARCA,MODELO
0,79293.0,18684.0,2015.0,4.0,19998.4,OWENSBORO,KY,DODGE,JourneySXT
1,190834.0,26318.0,2016.0,3.0,13563.2,CORPUS CHRISTI,TX,BMW,3
2,670383.0,25802.0,2014.0,5.0,105404.8,PORT LAVACA,TX,FORD,ExplorerLimited
3,586732.0,9899.0,2005.0,14.0,173619.2,LINDON,UT,DODGE,Dakota4WD
4,37609.0,30999.0,2016.0,3.0,13372.8,FOND DU LAC,WI,FORD,MustangFastback


## ANÁLISE DESCRITIVA

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

**Separando a base de dados em treinamento e teste aleatoriamente**

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

In [4]:
def dummify(data, column_name):
    return pd.get_dummies(data[column_name],drop_first = False, prefix = column_name)

def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y, X_cp)
    results = model.fit()
    ypred = results.predict(X_cp)
    return results, ypred

In [5]:
traindummy = pd.concat([dummify(dataset, "ESTADO").dropna(), dummify(dataset, "MODELO").dropna(), dummify(dataset, "MARCA").dropna()], axis=1)
dt = dataset.drop(columns = ["ID", "ANO", "CIDADE", "ESTADO", "MARCA", "MODELO"])
final = pd.concat([dt, traindummy], axis = 1)

In [6]:
train, test = train_test_split(final, test_size = 0.25, train_size = 0.75)

In [7]:
df = train

x = df[np.delete(df.columns.values, 0)]
y = df[["PRECO"]]

results = regress(x,y)

dataresultados = pd.read_html(results[0].summary().tables[1].as_html(), header=0, index_col=0)[0]

In [8]:
results[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PRECO   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.862
Method:                 Least Squares   F-statistic:                     227.1
Date:                Tue, 28 May 2019   Prob (F-statistic):               0.00
Time:                        16:18:39   Log-Likelihood:            -7.4160e+05
No. Observations:               75000   AIC:                         1.487e+06
Df Residuals:                   72930   BIC:                         1.506e+06
Df Model:                        2069                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                         4.941e+04    252.749    195.478      0.000    4.89e+04    4.99e+04
IDADE                        -1450.9020     10.569   -137.281      0.000   -1471.617   -1430.187
KM                              -0.0419      0.000    -92.764      0.000      -0.043      -0.041
ESTADO_ AK                     890.6407    427.507      2.083      0.037      52.728    1728.553
ESTADO_ AL                    1210.0155    148.181      8.166      0.000     919.580    1500.451
ESTADO_ AR                    1524.7327    203.909      7.478      0.000    1125.072    1924.394
ESTADO_ AZ                     639.4190    112.273      5.695      0.000     419.364     859.474
ESTADO_ CA                     705.5771     68.440     10.309      0.000     571.435     839.719
ESTADO_ CO                    1397.8927    115.908     12.060      0.000    1170.714    1625.071
ESTADO_ CT                     240.9234    165.685      1.454      0.146     -83.818     565.665
ESTADO_ DC                   -3.511e-05   7.33e-06     -4.790      0.000   -4.95e-05   -2.07e-05
ESTADO_ DE                     814.2689    310.062      2.626      0.009     206.548    1421.990
ESTADO_ FL                     -51.0682     69.183     -0.738      0.460    -186.667      84.531
ESTADO_ GA                     568.3538     89.038      6.383      0.000     393.839     742.869
ESTADO_ HI                    1421.1674    347.321      4.092      0.000     740.420    2101.915
ESTADO_ IA                    1389.5168    224.113      6.200      0.000     950.256    1828.777
ESTADO_ ID                    1242.2757    268.130      4.633      0.000     716.742    1767.809
ESTADO_ IL                     800.9237     89.303      8.969      0.000     625.891     975.957
ESTADO_ IN                     557.6483    129.021      4.322      0.000     304.767     810.530
ESTADO_ KS                     940.4203    184.693      5.092      0.000     578.423    1302.418
ESTADO_ KY                     240.1978    137.259      1.750      0.080     -28.829     509.225
ESTADO_ LA                     694.4412    181.314      3.830      0.000     339.066    1049.817
ESTADO_ MA                     746.0036    126.267      5.908      0.000     498.522     993.486
ESTADO_ MD                     384.9578    127.254      3.025      0.002     135.541     634.375
ESTADO_ ME                    1129.3206    400.066      2.823      0.005     345.194    1913.448
ESTADO_ MI                      33.3253    156.856      0.212      0.832    -274.111     340.762
ESTADO_ MN                    1129.4624    155.233      7.276      0.000     825.206    1433.719
ESTADO_ MO                     967.1927    129.972      7.442      0.000     712.447    1221.938
ESTADO_ MS                    1548.7185    205.888      7.522      0.000    

In [9]:
print(len(dataresultados), len(df))

2221 75000


In [10]:
for i in range(5):
    maiores = []
    for i in dataresultados.loc[(dataresultados["P>|t|"] > 0.10)].index.tolist():
        maiores.append(i)

    df = df.drop(columns = maiores)
    x = df[np.delete(df.columns.values, 0)]
    y = df[["PRECO"]]
    results = regress(x,y)
    dataresultados = pd.read_html(results[0].summary().tables[1].as_html(), header=0, index_col=0)[0]

In [34]:
print(len(dataresultados), len(df))

1385 75000


In [33]:
results[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PRECO   R-squared:                       0.861
Model:                            OLS   Adj. R-squared:                  0.858
Method:                 Least Squares   F-statistic:                     341.2
Date:                Tue, 28 May 2019   Prob (F-statistic):               0.00
Time:                        16:28:33   Log-Likelihood:            -7.4298e+05
No. Observations:               75000   AIC:                         1.489e+06
Df Residuals:                   73666   BIC:                         1.501e+06
Df Model:                        1333                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                         4.623e+04    179.310    257.825      0.000    4.59e+04    4.66e+04
IDADE                        -1438.1303      9.788   -146.930      0.000   -1457.314   -1418.946
KM                              -0.0417      0.000    -92.997      0.000      -0.043      -0.041
ESTADO_ AK                     957.6234    440.332      2.175      0.030      94.574    1820.673
ESTADO_ AL                    1240.8953    157.495      7.879      0.000     932.205    1549.585
ESTADO_ AR                    1606.2777    213.265      7.532      0.000    1188.280    2024.275
ESTADO_ AZ                     630.7581    121.575      5.188      0.000     392.471     869.045
ESTADO_ CA                     696.8910     79.743      8.739      0.000     540.595     853.187
ESTADO_ CO                    1395.6265    125.462     11.124      0.000    1149.721    1641.532
ESTADO_ DE                     812.2590    322.102      2.522      0.012     180.940    1443.578
ESTADO_ GA                     555.2826     99.122      5.602      0.000     361.004     749.561
ESTADO_ HI                    1318.9078    357.419      3.690      0.000     618.369    2019.447
ESTADO_ IA                    1441.1818    233.954      6.160      0.000     982.634    1899.730
ESTADO_ ID                    1191.9896    279.003      4.272      0.000     645.145    1738.834
ESTADO_ IL                     791.7181     99.480      7.959      0.000     596.738     986.699
ESTADO_ IN                     538.0264    138.413      3.887      0.000     266.738     809.315
ESTADO_ KS                     871.6276    194.340      4.485      0.000     490.721    1252.534
ESTADO_ KY                     250.2174    146.490      1.708      0.088     -36.902     537.337
ESTADO_ LA                     691.9856    190.664      3.629      0.000     318.285    1065.686
ESTADO_ MA                     741.8275    135.690      5.467      0.000     475.877    1007.778
ESTADO_ MD                     394.9640    136.544      2.893      0.004     127.339     662.589
ESTADO_ ME                    1182.0947    414.339      2.853      0.004     369.993    1994.197
ESTADO_ MN                    1149.1479    164.231      6.997      0.000     827.256    1471.040
ESTADO_ MO                     930.7661    139.269      6.683      0.000     657.800    1203.732
ESTADO_ MS                    1509.1267    215.380      7.007      0.000    1086.983    1931.271
ESTADO_ MT                    2055.6698    377.420      5.447      0.000    1315.929    2795.411
ESTADO_ NC                     944.6174    100.358      9.412      0.000     747.916    1141.318
ESTADO_ NE                    1564.7034    244.654      6.396      0.000    1085.183    2044.224
ESTADO_ NH                     847.6459    231.278      3.665      0.000    

In [13]:
quanti = 0
for i in dataresultados["P>|t|"]:
    if i > 0.10:
        quanti+=1
print("Quantidade de colunas com P>|t| maiores que alpha de 10%: ", quanti)

Quantidade de colunas com P>|t| maiores que alpha de 10%:  7


# TEST

In [16]:
for i in test.columns.tolist():
    if i not in df.columns.tolist():
        test = test.drop(columns = [i])

In [20]:
test.head()

,PRECO,IDADE,KM,ESTADO_ AK,ESTADO_ AL,ESTADO_ AR,ESTADO_ AZ,ESTADO_ CA,ESTADO_ CO,ESTADO_ DE,...,MARCA_SAAB,MARCA_SATURN,MARCA_SCION,MARCA_SUBARU,MARCA_SUZUKI,MARCA_TESLA,MARCA_TOYOTA,MARCA_VOLKSWAGEN,MARCA_VOLVO,MARCA_SMART
29597,37699.0,4.0,67520.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9047,9989.0,8.0,162195.2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73175,13752.0,6.0,104491.2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97862,29495.0,4.0,46304.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65550,52995.0,13.0,27700.8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
X_Test = sm.add_constant(test[np.delete(test.columns.values, 0)])
Y_pred_test = results[0].predict(X_Test).tolist()

In [50]:
rms_test_R = []
rms_train_R = []

testpreco = test["PRECO"].tolist()
trainpreco = train["PRECO"].tolist()

Y_pred_train = results[1].tolist()
Y_pred_test = results[0].predict(X_Test).tolist()


for a in range(len(trainpreco)):
    rms_train_R.append((trainpreco[a]-Y_pred_train[a])**2)

for e in range(len(testpreco)):
    rms_test_R.append((testpreco[e]-Y_pred_test[e])**2)


RMS_train_R = np.sqrt(np.mean(rms_train_R))
RMS_test_R = np.sqrt(np.mean(rms_test_R))
R_train_R = results[0].rsquared
R_test_R = r2_score(testpreco, Y_pred_test)

R_perc_R = ((R_test_R-R_train_R)/R_train_R)*100
RMS_perc_R = ((RMS_test_R-RMS_train_R)/RMS_train_R)*100

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS